In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader

In [4]:
#Obtain dataset
trainset = datasets.CIFAR10('./../0. Data/',
                               download = True,
                               train = True,
                               transform = T.Compose([
                                   T.ToTensor()
                               ]))

testset = datasets.CIFAR10('./../0. Data/',
                              download = True,
                              train = False,
                              transform = T.Compose([
                                  T.ToTensor()
                              ]))

#split training data further to training and validation data
train_set, val_set = torch.utils.data.random_split(trainset, [40000, 10000])

#Put in dataloader
train_loader_full = DataLoader(trainset,
                           batch_size=32,
                           shuffle=True)

train_loader = DataLoader(train_set,
                      batch_size = 32,
                      shuffle = True)

val_loader = DataLoader(val_set,
                       batch_size = 32,
                       shuffle = True)

test_loader = DataLoader(testset,
                        batch_size = 32,
                        shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#check for GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [13]:
#Create a MLP class defining our neural network
class vanilla_CNN(nn.Module):
    def __init__(self, input_len, output_len):
        super(vanilla_CNN, self).__init__()
        
        #Convvolutional and max pool layers
        self.conv2d_32 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), padding=1)
        self.batch2d_1 = nn.BatchNorm2d(32)
        self.conv2d_64 =  nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), padding=1)
        self.batch2d_2 = nn.BatchNorm2d(64)
        self.maxp2d_1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.conv2d_128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=1)
        self.batch2d_3 = nn.BatchNorm2d(128)
        self.maxp2d_2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2d_256 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding=1)
        self.batch2d_4 = nn.BatchNorm2d(256)
        self.maxp2d_3 = nn.MaxPool2d(kernel_size=2, stride=2)

        #three fully connected layers
        self.fc1 = nn.Linear(in_features=256*4*4, out_features=512)
        self.bn1 = torch.nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.bn2 = torch.nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)
        
        
    def forward(self, x):
        x = x.view(-1,3,32,32)
        #input_size = 1x28x28, output_size = 32x28x28
        x = F.relu(self.conv2d_32(x))
        #input_size = 32x28x28, output_size = 32x28x28
        x = self.batch2d_1(x)
        #input_size = 32x28x28, output_size = 64x28x28
        x = F.relu(self.conv2d_64(x))
        #input_size = 64x28x28, output_size = 64x28x28
        x = self.batch2d_2(x)
        #input_size = 64x28x28, output_size = 64x15x15
        x = F.relu(self.maxp2d_1(x))
        #input_size = 64x15x15, output_size = 128x15x15
        x = F.relu(self.conv2d_128(x))
        #input_size = 128x15x15, output_size = 128x15x15
        x = self.batch2d_3(x)
        #input_size = 128x15x15, output_size = 128x7x7
        x = F.relu(self.maxp2d_2(x))
        #input_size = 128x7x7, output_size = 256x7x7
        x = F.relu(self.conv2d_256(x))
        #input_size = 256x7x7, output_size = 256x7x7
        x = self.batch2d_4(x)
        #input_size = 256x7x7, output_size = 256x3x3
        x = F.relu(self.maxp2d_3(x))

        #convert image to a one dimentional tensor before feeding to neural network
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.fc3(x)
        
        return x
        

In [14]:
#define training function
def train(Model, validate, max_epoch):
    for epoch in range(max_epoch):
        Train_Loss = []
        Val_Loss =[]
        loader = train_loader_full
        
        if(validate):
            loader = train_loader
        
        cnf_tr = torch.zeros(10,10)
        cnf_val = torch.zeros(10,10)
        
        #Train on training data
        for i, sample in enumerate(loader):

            #set model to train mode
            Model.train()
            #set gradiuents to zero
            optimizer.zero_grad()
            #obtain output
            output = Model(sample[0].to(device)).to(device)
            #compute loss
            loss = loss_function(output, sample[1].to(device))
            #compute gradients
            loss.backward()
            #optimize weights
            optimizer.step()
            #record train loss
            Train_Loss.append(loss.item())
            
            with torch.no_grad():
                #calculate output by argmax
                output = torch.argmax(output, 1)
                #update entries in confusion matrix
                for i in range(output.shape[0]):
                    cnf_tr[output[i],sample[1][i]] +=1
            
        if(validate):
            #Evaluate on validation data
            with torch.no_grad():
                #set model to evaluation mode
                Model.eval()
                #evaluate on tvaidation data
                for i, sample in enumerate(val_loader):
                    output = Model(sample[0].to(device))
                    loss = loss_function(output, sample[1].to(device))
                    Val_Loss.append(loss.item())
                    #calculate output by argmax
                    output = torch.argmax(output, 1)
                    #update entries in confusion matrix
                    for i in range(output.shape[0]):
                        cnf_val[output[i],sample[1][i]] +=1
                   
        actual_count = torch.sum(cnf_tr, dim=0)
        correct_pred = torch.tensor([cnf_tr[i,i] for i in range(10)])
        A_tr = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        if(validate):
            actual_count = torch.sum(cnf_val, dim=0)
            correct_pred = torch.tensor([cnf_val[i,i] for i in range(10)])
            A_val = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        #print losses in every epoch
        if(validate):
            print('epoch : ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Val_acc : ', np.round(A_val,4),  
                  '; Train_loss  ',np.round(np.mean(Train_Loss),4),  '; Val_loss  ',np.round(np.mean(Val_Loss),4))
        else:
            print('epoch = ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Train_loss  ',np.round(np.mean(Train_Loss),4))

In [15]:
#Function top evaluate model using performace metrices
def evaluate(cnf):
    actual_count = torch.sum(cnf, dim=0)
    predicted_count = torch.sum(cnf, dim=1)
    correct_pred = torch.tensor([cnf[i,i] for i in range(10)])
    #Precision
    precision = correct_pred/predicted_count
    #Recall
    recall = correct_pred/actual_count
    #F1-Score
    f1_score = 2*precision*recall/(precision+recall)
    #Accuracy
    Accuracy = torch.sum(correct_pred)/torch.sum(actual_count)
    print('\n',pd.DataFrame({'Class':[i for i in range(10)],
                 'Precision' : precision,
                 'Recall' : recall,
                 'F1_Score': f1_score}))
    
    
    print('\nAccuracy  : ', Accuracy.item())

In [16]:
#function to test model
def test(Model):
    Loss = []
    #confusion matrix
    cnf = torch.zeros(10,10)

    #evaluate on test data
    with torch.no_grad():
        #set model to evaluation mode
        Model.eval()
        #evaluate on test data
        for i, sample in enumerate(test_loader):
            output = Model(sample[0].to(device))
            loss = loss_function(output, sample[1].to(device))
            Loss.append(loss.item())
            #calculate output by argmax
            output = torch.argmax(output, 1)
            #update entries in confusion matrix
            for i in range(output.shape[0]):
                cnf[output[i],sample[1][i]] +=1

        #print test loss
        print('Test loss : ', np.mean(Loss))

    #print evaluation summary
    evaluate(cnf)

In [17]:
#define loss function
loss_function = nn.CrossEntropyLoss()

In [18]:
#Create Model
Model = vanilla_CNN(1024,10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train model with validation
train(Model, validate=True, max_epoch=20)

epoch :  0 ; Train_acc :  0.5553 ; Val_acc :  0.6479 ; Train_loss   1.2407 ; Val_loss   1.0125
epoch :  1 ; Train_acc :  0.7149 ; Val_acc :  0.7424 ; Train_loss   0.8207 ; Val_loss   0.7603
epoch :  2 ; Train_acc :  0.7622 ; Val_acc :  0.6943 ; Train_loss   0.6898 ; Val_loss   0.876
epoch :  3 ; Train_acc :  0.8023 ; Val_acc :  0.7899 ; Train_loss   0.5726 ; Val_loss   0.6343
epoch :  4 ; Train_acc :  0.8413 ; Val_acc :  0.7893 ; Train_loss   0.4647 ; Val_loss   0.6352
epoch :  5 ; Train_acc :  0.8622 ; Val_acc :  0.8016 ; Train_loss   0.3974 ; Val_loss   0.6364
epoch :  6 ; Train_acc :  0.8905 ; Val_acc :  0.789 ; Train_loss   0.3163 ; Val_loss   0.7497
epoch :  7 ; Train_acc :  0.9075 ; Val_acc :  0.7119 ; Train_loss   0.2687 ; Val_loss   1.1045
epoch :  8 ; Train_acc :  0.9128 ; Val_acc :  0.7953 ; Train_loss   0.249 ; Val_loss   0.7921
epoch :  9 ; Train_acc :  0.9381 ; Val_acc :  0.7992 ; Train_loss   0.1778 ; Val_loss   0.7935
epoch :  10 ; Train_acc :  0.9427 ; Val_acc :  0.7963

In [20]:
#Let's train our model for 10 epochs on full training set
#Create Model
Model = vanilla_CNN(1024,10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train
train(Model, validate=False, max_epoch=10)

epoch =  0 ; Train_acc :  0.5633 ; Train_loss   1.2346
epoch =  1 ; Train_acc :  0.7041 ; Train_loss   0.8594
epoch =  2 ; Train_acc :  0.7539 ; Train_loss   0.7184
epoch =  3 ; Train_acc :  0.8041 ; Train_loss   0.5735
epoch =  4 ; Train_acc :  0.8328 ; Train_loss   0.4861
epoch =  5 ; Train_acc :  0.859 ; Train_loss   0.4128
epoch =  6 ; Train_acc :  0.8897 ; Train_loss   0.3255
epoch =  7 ; Train_acc :  0.9087 ; Train_loss   0.2666
epoch =  8 ; Train_acc :  0.9138 ; Train_loss   0.2486
epoch =  9 ; Train_acc :  0.9203 ; Train_loss   0.233


In [21]:
test(Model)

Test loss :  0.6614584162974129

    Class  Precision  Recall  F1_Score
0      0   0.847890   0.864  0.855869
1      1   0.914286   0.896  0.905051
2      2   0.775269   0.721  0.747150
3      3   0.632727   0.696  0.662857
4      4   0.743847   0.816  0.778255
5      5   0.742976   0.714  0.728200
6      6   0.827947   0.871  0.848928
7      7   0.914481   0.802  0.854555
8      8   0.909369   0.893  0.901110
9      9   0.885230   0.887  0.886114

Accuracy  :  0.8159999847412109


In [22]:
#Finally, let's save our model
torch.save(Model.state_dict(), './saved_models/vanilla_CNN_CIFAR10.pth')

In [23]:
#To Retrieve
Modelx = vanilla_CNN(1024,10)
Modelx.load_state_dict(torch.load('./saved_models/vanilla_CNN_CIFAR10.pth'))

<All keys matched successfully>